In [ ]:
import numpy as np
from IPython.core.debugger import set_trace
import re
import difflib
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import time
import random
import pickle
from lxml.html import fromstring
from collections import Counter
from requests.packages.urllib3.util import Retry
from requests.adapters import HTTPAdapter
from requests import Session, exceptions
import sys
import pymongo
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import os
import json
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
from nltk.tokenize import sent_tokenize, word_tokenize
sys.path.append("/Users/rohanramesh/Documents/GitHub/Insight_writers/lib/")
from text_processing import ProcessArticle as pa
import suggestions as s
import difflib
from scipy.stats import wilcoxon, sem

In [ ]:
# data load
with open('/Users/rohanramesh/Documents/Insight/data_bball_writers/Attempt3_mr_scrape.pickle', 'rb') as handle:
    scrapevar = pickle.load(handle)
    
# load writer df
with open('/Users/rohanramesh/Documents/Insight/data_bball_writers/writer_df.pickle', 'rb') as handle:
    writer_df = pickle.load(handle)
    
# load writer features
writer_features = pd.read_pickle('/Users/rohanramesh/Documents/Insight/data_bball_writers/writer_features.pickle')

# load podcasts
top_podcasts = pd.read_pickle('/Users/rohanramesh/Documents/Insight/data_bball_writers/top50podcasts.pickle')


In [ ]:
dbname = 'writer_feature_db'
# dbname = 'writer_db'
username = 'rohanramesh' # change this to your username
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))
con = None
con = psycopg2.connect(database = dbname, user = username)

In [ ]:
# query to load entire db into pandas
sql_query = """
SELECT * FROM writer_feature;
"""
writer_feature_df = pd.read_sql_query(sql_query,con)

In [ ]:
sql_query = """
SELECT AVG("n_words") AS "avg_n_words",
         AVG("neg_sent") AS "avg_neg_sent",
         AVG("neu_sent") AS "avg_neu_sent",
         AVG("pos_sent") AS "avg_pos_sent",
         AVG("neg_sent_var") AS "avg_neg_sent_var",
         AVG("neu_sent_var") AS "avg_neu_sent_var",
         AVG("pos_sent_var") AS "avg_pos_sent_var",
        AVG("n_sentences") AS "avg_n_sentences",
        AVG("n_wordspersentence") AS "avg_n_wordspersentence",
        AVG("n_wordspersent_variability") AS "avg_n_wordspersent_variability",
        AVG("wordlength") AS "avg_wordlength",
        AVG("wordlength_var") AS "avg_wordlength_var",
        AVG("wordlength_skew") AS "avg_wordlength_skew",
        "author_list"
FROM writer_feature
WHERE "n_words" > 100
GROUP BY "author_list"
HAVING COUNT(*) > 25 
ORDER BY AVG("n_words") DESC
"""
writer_feature_subsection = pd.read_sql_query(sql_query,con)
writer_feature_subsection.head(10)
writer_feature_subsection.replace([np.inf, -np.inf], np.nan)
writer_feature_subsection = writer_feature_subsection.dropna(axis=0, how='any')

In [ ]:
author_webnames = writer_feature_subsection['author_list']
author_list = []
for i in author_webnames:
    idx = writer_df['website_name'] == i
#     print(i)
#     print(writer_df['Idea Text'][np.where(idx)[0]].tolist()[0])
#     break
    author_list.append(writer_df['Idea Text'][np.where(idx)[0]].tolist()[0])
    
# author_list[41] = 'Jason Concepcion'

In [ ]:
# type something in search key
# new_id = url_format.format('zach', 'lowe')
# new_id = 'https://www.listennotes.com/search/?q=zach%20lowe'
page = requests.get(url_format)
curr_soup = BeautifulSoup(page.text)

In [ ]:
# top_podcasts = {}
# for curr_author in author_list:
#     top_podcasts[curr_author] = []
#     r = word_tokenize(curr_author)
#     page_idx = 0
#     while page_idx < 50:
# #         url = ("https://listennotes.p.mashape.com/api"
# #             "/v1/search?offset={}&q={}+{}&sort_by_date=0&type=episode".format(page_idx,r[0], r[1]))
# #         response = requests.get(url,
# #           headers={
# #             "X-Mashape-Key": "PUT IN KEY",
# #             "Accept": "application/json"
# #           }
# #         )
#         a = response.content
#         b = json.loads(a)
#         top_podcasts[curr_author].append(b)
#         page_idx += 10
#         time.sleep(random.uniform(0.5,2))
#         print(url)


In [ ]:
# These code snippets use an open-source library. http://unirest.io/python
response = requests.get("https://listennotes.p.mashape.com/api/v1/search?offset=0&q=zach+lowe&sort_by_date=0&type=episode",
  headers={
    "X-Mashape-Key": "KEYINEMAIL",
    "Accept": "application/json"
  }
)

In [ ]:
# print(ast.literal_eval(a))
a = response.content
b = json.loads(a)

In [ ]:
# for each podcast lets identify fields to search
author_mentions = np.zeros([len(author_list), len(author_list)])
for i in range(0,len(author_list)):
    curr_author = author_list[i]
    # curr_page = 0
    for curr_page in range(0,4):
    # curr_pod = 0
        for curr_pod in range(0,np.min([9,len(top_podcasts[curr_author][curr_page]['results'])])):
            curr_info = top_podcasts[curr_author][curr_page]['results'][curr_pod]
            podcast_keys = curr_info.keys()
            matches = []
            for curr_key in podcast_keys:
                for j in range(0,len(author_list)):
                    curr_authorcomp = author_list[j]
                    if curr_authorcomp == 'Jason Concepcion/netw3rk':
                        curr_authorcomp = 'Jason Concepcion'
                    search_terms = curr_authorcomp
                    #     matches = []
                    if isinstance(curr_info[curr_key], str):
                #         print(curr_info[curr_key])
                        last_name = word_tokenize(search_terms)[1]
                #         print(last_name)
                    #     for word in search_terms.split(" "):
                    #         print(word)
                        if last_name in curr_info[curr_key]:
                            author_mentions[i,j] = 1
                        

In [ ]:
# curr_info
# top_podcasts[curr_author][curr_page]
# np.min([9,len(top_podcasts[curr_author][curr_page]['results'])])
def match_author_names(writer_df, author):
    wa = writer_df['website_name'].tolist()
    al = writer_df['Idea Text'].tolist()
    if author in wa:
        return al[wa.index(author)]
    elif author in al:
        return wa[al.index(author)]
    else:
        print('improper author name')
            

In [ ]:
plt.imshow(author_mentions)

In [ ]:
r = 'Zach Lowe'
idx = np.where(author_mentions[author_list.index(r)])
b = [author_list[i] for i in idx[0] if author_list[i] != r]
b

In [ ]:
author_sugg = s.give_author_suggestion_from_author(writer_features, 'zachlowe_nba')
print(author_sugg['authors'].tolist())

In [ ]:
# iterate each author and see how many times match in podcasts in either direction
# for i in range(0,len(author_list)):
website_author_names = writer_df['website_name'].tolist()
matched_fraction = []
nonmatch_fraction = []
for ii in range(0,len(author_list)):
# ii = 7
    if np.sum(author_mentions[ii,:]) <= 1:
        print(author_list[ii])
        continue
    curr_author = author_list[ii]
    website_name = match_author_names(writer_df, curr_author)
    author_sugg = s.give_author_suggestion_from_author(writer_features, website_name)
    author_sugg_list = author_sugg['authors'].tolist()
    matched_authors = author_sugg_list[0:6]
#     print(matched_authors)
    nonmatched_authors = [i for i in website_author_names if i not in matched_authors]
    # to remove same author
    matched_authors = matched_authors[1:]
    # get n for matched authors
    n_match = []
    for i in matched_authors:
        idx1 = author_list.index(curr_author)
        idx2 = author_list.index(match_author_names(writer_df, i))
        val = np.max([author_mentions[idx1,idx2], author_mentions[idx2,idx1]])
        n_match.append(val)
    matched_fraction.append(np.sum(n_match)/len(n_match))
    n_nonmatch = []
    for i in nonmatched_authors:
        if match_author_names(writer_df, i) in author_list:
            idx1 = author_list.index(curr_author)  
            idx2 = author_list.index(match_author_names(writer_df, i))
            val = np.max([author_mentions[idx1,idx2], author_mentions[idx2,idx1]])
            n_nonmatch.append(val)
    nonmatch_fraction.append(np.sum(n_nonmatch)/len(n_nonmatch))


In [ ]:
plt.hist(nonmatch_fraction)
plt.show()
plt.hist(matched_fraction)
plt.show()
print(np.mean(matched_fraction))
print(np.mean(nonmatch_fraction))
# (np.mean(matched_fraction)-np.mean(nonmatch_fraction))/np.mean(matched_fraction)
# print((np.mean(nonmatch_fraction)*100)/(np.mean(matched_fraction)*100-np.mean(nonmatch_fraction)*100))
print(np.mean(matched_fraction)/np.mean(nonmatch_fraction))
a,pval = wilcoxon(matched_fraction, nonmatch_fraction)
pval

In [ ]:
# plt.boxplot([matched_fraction, nonmatch_fraction])
plt.bar(range(0,2),[np.mean(matched_fraction)*100, np.mean(nonmatch_fraction)*100], 0.75, 
        yerr=[sem(matched_fraction)*100, sem(nonmatch_fraction)*100], tick_label= ['Suggested', 'Not suggested'])
plt.ylabel('Percent of writers to do podcasts together')
pval

In [ ]:
curr_page = 2
curr_author = 'Zach Lowe'
curr_pod = 8
# for curr_pod in range(0,np.min([9,len(top_podcasts[curr_author][curr_page]['results'])])):
curr_info = top_podcasts[curr_author][curr_page]['results'][curr_pod]
podcast_keys = curr_info.keys()
matches = []
for curr_key in podcast_keys:
    for j in range(0,len(author_list)):
        curr_authorcomp = author_list[j]
        search_terms = curr_authorcomp
        #     matches = []
        if isinstance(curr_info[curr_key], str):
#             print(curr_info[curr_key])
            last_name = word_tokenize(search_terms)[1]
    #         print(last_name)
        #     for word in search_terms.split(" "):
        #         print(word)
            if last_name in curr_info[curr_key]:
    #                 author_mentions[i,j] = 1
                print(curr_authorcomp)
                print('woo')
                